# Problème d'allocation de ressources avec algorithme génétique

In [37]:
import numpy as np
import numpy.random as rd
from numpy.random import rand

L'enjeu de ce projet est de résoudre un problème classique d'allocation de ressources consistant à répartir $N$ objets dans $M$ emplacements de façon à minimiser une certaine fonction de coût.

Notre problème prend la forme d'un graphe comprenant un ensemble de $n$ sommets ($S=\{s_1,...s_n\}$) reliés par des arêtes portant chacune une valeur numérique appelée poids. Le graphe considéré est complet, c'est-à-dire que ses sommets sont tous reliés deux à deux : on a donc $\frac{n(n-1)}{2}$ arêtes et autant de poids.

Le graphe peut être représenté par une matrice carré de taille $n$, appelée matrice d'adjacence, dans laquel la valeur située au croisement de la ligne $i$ et de la colonne $j$ correspond au poids de l'arête reliant $s_i$ à $s_j$. Par convention, les valeurs diagonales sont égales à 1.

Le poids $P(s_i)$ du sommet $s_i$ est défini comme le produit des composantes de la ligne $i$ de la matrice d'adjacence tandis que la fonction de coût (l'énergie totale) correspond à la somme des poids de tous les sommets du graphe.

Le but de l'exercice est de trouver un placement des $n$ premiers nombres premiers dans le graphe qui minimise l'énergie totale pour $n = 4, 5, 6, 7$.

Pour ce faire, il est inconcevable d'explorer exhaustivement les $\frac{n(n-1)}{2}!$ permutations possibles (on dépasse le trillion de possibilités dès $n=6$). On va donc utiliser un algorithme génétique simplifié, avec mutation aléatoire mais sans croisement, pour approcher le minimum de la fonciton de coût en recourant à moins de calculs.

## Préparation des fonctions

In [38]:
# Création d'une fonction générant une liste des premiers nombres premiers pour un graphe de taille n
def premiers(n):
    # On calcule le nombre de premiers nécessaires pour ensuite obtenir une matrice d'adjacence de taille n
    N = (n*(n-1)/2) 
    L = []
    candidat = 2
    # On fait en sorte d'obtenir une liste de premiers de taille N
    while len(L) < N:
        i = 2
        premier = True
    # On parcourt les entiers naturels en testant la primalité de chacun d'eux
        while i <= np.sqrt(candidat) and premier == True:
            if candidat % i == 0:
                premier = False
            i += 1
        if premier == True:
            L.append(candidat)
        candidat += 1
    return L

# Test
premiers(4)

[2, 3, 5, 7, 11, 13]

In [39]:
# Création d'une fonction générant une population initiale de permutations
def population(taille, n):
    pop = []
    for _ in range(taille):
        pop.append(rd.permutation(premiers(n)))
    return pop

# Test
pop = population(5, 4)
pop

[array([ 3, 11,  2,  7,  5, 13]),
 array([13,  2,  7, 11,  5,  3]),
 array([ 2, 13, 11,  7,  5,  3]),
 array([13,  3,  2, 11,  5,  7]),
 array([ 2, 13,  3,  5, 11,  7])]

In [40]:
# Création d'une fonction de mutation : chaque élément de chaque individu de la population a une chance d'échanger aléatoirement sa valeur avec un autre.
def mutation(population):
    for ind in population:
        for j in range(len(ind)):
            if rand() < 1/len(ind):
        # Si l'élément courant passe le test, on choisit une valeur aléatoire dans la liste des nombres premiers / valeurs possibles
                mut = rd.choice(ind) 
        # L'élement qui portait jusqu'alors la valeur sélectionnée prend la valeur de l'élément courant...
                ind[np.where(ind == mut)] = ind[j]
        # ...et l'élement courant prend sa nouvelle valeur.
                ind[j] = mut
 
# Test
mutation(pop)
pop

[array([ 3, 11,  2,  5,  7, 13]),
 array([13,  5,  7, 11,  2,  3]),
 array([ 2, 13, 11,  7,  5,  3]),
 array([13,  3,  7, 11,  2,  5]),
 array([ 2, 13,  3,  5, 11,  7])]

In [41]:
# Création d'une fonction générant la matrice d'adjacence correspondant à la liste de premiers passée en paramètre
def mat_adj(l_premiers):
    n = int((1 + np.sqrt(1 + 8 * len(l_premiers)))/ 2) #Permet de retrouver la taille de la matrice à partir de celle de la taille de la liste des premiers
    mat = np.zeros((n,n))
    ind = 0
    for i in range(n-1):
        for j in range(i+1,n):
            mat[i,j] = l_premiers[ind]
            ind += 1
    return mat + mat.T + np.eye(n)

# Test
mat_adj(pop[0])

array([[ 1.,  3., 11.,  2.],
       [ 3.,  1.,  5.,  7.],
       [11.,  5.,  1., 13.],
       [ 2.,  7., 13.,  1.]])

In [42]:
# Fonction calculant l'énergie d'une permutation donnée : c'est cette énergie qu'on cherchera à minimiser
def energie(l_premiers):
    M = mat_adj(l_premiers)
    n = np.shape(M)[1]
    e = 0
    # A chaque tour, on prend une des n colonnes, calcule le produit de ses composantes et l'ajoute au total
    for i in range(n):
        e += np.product(M[:,i])
    return e

# Test
energie(pop[0])

1068.0

In [43]:
# Création d'une fonction permettant d'obtenir une nouvelle génération de permutations
def cycle_de_vie(pop):
    mutation(pop)  
    # Après avoir muté la population initiale, on calcule l'énergie / le coût lié à chacune des permutations qu'elle contient
    fitness = [energie(ind) for ind in pop]
    const_norm = np.sum(fitness)
    # On calcule la probabilité pour chaque permutation d'être sélectionnée : dans le cadre de notre problème (minimisation), 
    # celle-ci augmente lorsque l'énergie dépensée est relativement faible et inversement.
    prob_selec = [1 - (fit/const_norm) for fit in fitness]
    nouvelle_gen = []
    for i in range(len(pop)):
    # En utilisant les probabilité de sélection individuelles, on sélectionne aléatoirement les permutations qui survivront à la prochaine génération
        if rand() < prob_selec[i]:
            nouvelle_gen.append(pop[i])
    # Pour conserver la taille de la population, on la complète avec un nombre suffisant de permutations aléatoires ; celles-ci permettront d'explorer
    # d'autres pistes
    for i in range(len(pop) - len(nouvelle_gen)):
        nouvelle_gen.append(rd.permutation(premiers((1 + np.sqrt(1 + 8 * len(pop[0])))/ 2)))
    return nouvelle_gen

# Test
pop
cycle_de_vie(pop)

[array([ 3, 11,  7,  5,  2, 13]),
 array([ 2,  5,  3, 13, 11,  7]),
 array([ 2,  7,  3, 11, 13,  5]),
 array([ 3,  5,  7,  2, 11, 13]),
 array([ 2,  7,  3, 13,  5, 11])]

La fonction précédente constitue le coeur de l'algorithme génétique.

On part d'une petite "population" de permutations des $n$ premiers nombres premiers, puis on calcule l'énergie obtenue pour chacune d'elle. 

Le score de __fitness__ ainsi obtenu conditionne la probabilité pour chaque permutation d'être sélectionnée pour la génération suivante : plus l'énergie est basse, plus la probabilité est élevée. 

Les permutations qui ne sont pas retenues "mutent" aléatoirement, c'est-à-dire qu'elles sont remplacées dans la génération suivante par de nouvelles permutations.



## Application de l'algorithme génétique pour n = 4, 5, 6 ou 7

In [44]:
# Cas où n = 4
# Nombre de premiers nécessaires : 6
# Nombre de permutations possibles : 6! = 720

n = 4
pop = population(10, n)
best_permutation = premiers(n)
best_fit_ever = energie(best_permutation)

for i in range(20):
    pop = cycle_de_vie(pop)
    fit = [energie(ind) for ind in pop]
    best_fit_act = min(fit)
    if best_fit_act < best_fit_ever:
        best_fit_ever = best_fit_act
        best_permutation = pop[fit.index(best_fit_act)].copy()
        print('Génération {}  MinEnergie = {}  Permutation : {}'.format(i, best_fit_ever, best_permutation))
print("\nMatrice d'adjacence correspondant à la meilleure permutation:\n {}".format(mat_adj(best_permutation)))

Génération 0  MinEnergie = 828.0  Permutation : [ 5  3 13 11  7  2]
Génération 2  MinEnergie = 782.0  Permutation : [ 7  2  5 11  3 13]
Génération 4  MinEnergie = 718.0  Permutation : [ 2 11  5  7 13  3]

Matrice d'adjacence correspondant à la meilleure permutation:
 [[ 1.  2. 11.  5.]
 [ 2.  1.  7. 13.]
 [11.  7.  1.  3.]
 [ 5. 13.  3.  1.]]


In [45]:
# Cas où n = 5
# Nombre de premiers nécessaires : 10
# Nombre de permutations possibles : 10! = 3 628 800

n = 5
pop = population(10, n)
best_permutation = premiers(n)
best_fit_ever = energie(best_permutation)

for i in range(6000):
    pop = cycle_de_vie(pop)
    fit = [energie(ind) for ind in pop]
    best_fit_act = min(fit)
    if best_fit_act < best_fit_ever:
        best_fit_ever = best_fit_act
        best_permutation = pop[fit.index(best_fit_act)].copy()
        print('Génération {}  MinEnergie = {}  Permutation : {}'.format(i, best_fit_ever, best_permutation))
print("\nMatrice d'adjacence correspondant à la meilleure permutation:\n {}".format(mat_adj(best_permutation)))

Génération 0  MinEnergie = 54171.0  Permutation : [19 11  7 17 29 13  2  3  5 23]
Génération 7  MinEnergie = 50385.0  Permutation : [ 7 11 23  3 19 17  5  2 13 29]
Génération 16  MinEnergie = 49269.0  Permutation : [ 3  5 23 29 19 11  7 17 13  2]
Génération 25  MinEnergie = 47039.0  Permutation : [ 5 29 17  3 11  7 13  2 19 23]
Génération 27  MinEnergie = 46621.0  Permutation : [ 5 29 17  3 13  7 11  2 19 23]
Génération 41  MinEnergie = 45987.0  Permutation : [ 5  2 13 29 23 11  7 19 17  3]
Génération 42  MinEnergie = 43035.0  Permutation : [ 5  2 19 29 23 11  7 13 17  3]
Génération 311  MinEnergie = 42581.0  Permutation : [17 11  7  5 13  2 23 19  3 29]
Génération 1161  MinEnergie = 42207.0  Permutation : [ 5 29  3 23  7 13 17 19  2 11]

Matrice d'adjacence correspondant à la meilleure permutation:
 [[ 1.  5. 29.  3. 23.]
 [ 5.  1.  7. 13. 17.]
 [29.  7.  1. 19.  2.]
 [ 3. 13. 19.  1. 11.]
 [23. 17.  2. 11.  1.]]


In [46]:
# Cas où n = 6
# Nombre de premiers nécessaires : 15
# Nombre de permutations possibles : 15! = 1 307 674 368 000

n = 6
pop = population(10, n)
best_permutation = premiers(n)
best_fit_ever = energie(best_permutation)

for i in range(30000):
    pop = cycle_de_vie(pop)
    fit = [energie(ind) for ind in pop]
    best_fit_act = min(fit)
    if best_fit_act < best_fit_ever:
        best_fit_ever = best_fit_act
        best_permutation = pop[fit.index(best_fit_act)].copy()
        print('Génération {}  MinEnergie = {}'.format(i, best_fit_ever))
print("\nEnergie minimale = {}\nPermutation optimale estimée :\n {}\nMatrice d'adjacence correspondante:\n{}".format(best_fit_ever, best_permutation, mat_adj(best_permutation)))

Génération 0  MinEnergie = 8404698.0
Génération 4  MinEnergie = 6910922.0
Génération 28  MinEnergie = 6504632.0
Génération 31  MinEnergie = 5343332.0
Génération 677  MinEnergie = 5156132.0
Génération 11151  MinEnergie = 5130944.0

Energie minimale = 5130944.0
Permutation optimale estimée :
 [41 13 17  3 29 43  2 47  5 23 11  7 31 37 19]
Matrice d'adjacence correspondante:
[[ 1. 41. 13. 17.  3. 29.]
 [41.  1. 43.  2. 47.  5.]
 [13. 43.  1. 23. 11.  7.]
 [17.  2. 23.  1. 31. 37.]
 [ 3. 47. 11. 31.  1. 19.]
 [29.  5.  7. 37. 19.  1.]]


In [47]:
# Cas où n = 7
# Nombre de premiers nécessaires : 21
# Nombre de permutations possibles : 21! = 51 090 942 171 709 440 000

n = 7
pop = population(10, n)
best_permutation = premiers(n)
best_fit_ever = energie(best_permutation)

for i in range(50000):
    pop = cycle_de_vie(pop)
    fit = [energie(ind) for ind in pop]
    best_fit_act = min(fit)
    if best_fit_act < best_fit_ever:
        best_fit_ever = best_fit_act
        best_permutation = pop[fit.index(best_fit_act)].copy()
        print('Génération {}  MinEnergie = {}'.format(i, best_fit_ever))
print("\nEnergie minimale = {}\nPermutation optimale estimée :\n {}\nMatrice d'adjacence correspondante:\n{}".format(best_fit_ever, best_permutation, mat_adj(best_permutation)))

Génération 0  MinEnergie = 1535902163.0
Génération 10  MinEnergie = 1274407961.0
Génération 254  MinEnergie = 1263368997.0
Génération 272  MinEnergie = 1222767199.0
Génération 426  MinEnergie = 1194451175.0
Génération 729  MinEnergie = 1170408275.0
Génération 730  MinEnergie = 1144843807.0
Génération 886  MinEnergie = 1116051717.0
Génération 4197  MinEnergie = 1114793241.0
Génération 4384  MinEnergie = 1069465803.0

Energie minimale = 1069465803.0
Permutation optimale estimée :
 [11  7 31 73 37 17  5 43 61 19 59 23 67 41 71  2 53 47 29 13  3]
Matrice d'adjacence correspondante:
[[ 1. 11.  7. 31. 73. 37. 17.]
 [11.  1.  5. 43. 61. 19. 59.]
 [ 7.  5.  1. 23. 67. 41. 71.]
 [31. 43. 23.  1.  2. 53. 47.]
 [73. 61. 67.  2.  1. 29. 13.]
 [37. 19. 41. 53. 29.  1.  3.]
 [17. 59. 71. 47. 13.  3.  1.]]
